In [41]:
from langchain.chains import SequentialChain, LLMChain
from langchain.memory import SimpleMemory
from langchain.llms.fake import FakeListLLM
from langchain.prompts import PromptTemplate

# Define your prompt templates
summary_template = """Write a summary of the following podcast text: 
{transcript}
SUMMARY :"""
guest_template = """Write a summary of the following podcast text as if you are the guest(s) posting on social media.
{transcript}
SUMMARY :"""
host_template = """Write a summary of the following podcast text as if you are the host posting on social media.
{transcript}
SUMMARY :"""

# Create PromptTemplates
SUMMARY_PROMPT = PromptTemplate(template=summary_template, input_variables=["transcript"])
GUEST_PROMPT = PromptTemplate(template=guest_template, input_variables=["transcript"])
HOST_PROMPT = PromptTemplate(template=host_template, input_variables=["transcript"])

# LLM to use in each stage 
responses = ["Response 1", "Response 2", "Response 3"]
llm = FakeListLLM(responses=responses)

# Create LLMChains for each stage
summary_chain = LLMChain(llm=llm, prompt=SUMMARY_PROMPT, output_key = 'summary')
guest_chain = LLMChain(llm=llm, prompt=GUEST_PROMPT, output_key = 'guest_post')
host_chain = LLMChain(llm=llm, prompt=HOST_PROMPT, output_key= 'self_post')


overall_chain = SequentialChain(
    memory=SimpleMemory(memories = {}),
    chains=[summary_chain, guest_chain, host_chain],
    input_variables=['transcript'],
    output_variables=["summary", "guest_post", "self_post"],
    verbose=True)

results = overall_chain({"transcript": "Your podcast transcript here"})

# Print the results
print(results)



> Entering new SequentialChain chain...

> Finished chain.
{'transcript': 'Your podcast transcript here', 'text5': 'Your podcast transcript here', 'summary': 'Response 1', 'guest_post': 'Response 2', 'self_post': 'Response 3'}


In [44]:
from langchain.chains import SequentialChain, LLMChain
from langchain.memory import SimpleMemory
from langchain.llms.fake import FakeListLLM
from langchain.prompts import PromptTemplate

# Define your prompt templates
summary_template = """Write a summary of the following podcast text: 
{transcript}
SUMMARY :"""
guest_template = """Write a summary of the following podcast text as if you are the guest(s) posting on social media.
{summary}
SUMMARY :"""
host_template = """Write a summary of the following podcast text as if you are the host posting on social media.
{transcript}
SUMMARY :"""

# Create PromptTemplates
SUMMARY_PROMPT = PromptTemplate(template=summary_template, input_variables=["transcript"])
GUEST_PROMPT = PromptTemplate(template=guest_template, input_variables=["transcript"])
HOST_PROMPT = PromptTemplate(template=host_template, input_variables=["transcript"])

# LLM to use in each stage 
responses = ["Response 1", "Response 2", "Response 3"]
llm = FakeListLLM(responses=responses)

# Create LLMChains for each stage
summary_chain = LLMChain(llm=llm, prompt=SUMMARY_PROMPT, output_key = 'summary')
guest_chain = LLMChain(llm=llm, prompt=GUEST_PROMPT, output_key = 'guest_post')
host_chain = LLMChain(llm=llm, prompt=HOST_PROMPT, output_key= 'self_post')


overall_chain = SequentialChain(
    # memory=SimpleMemory(memories = {}),
    chains=[summary_chain, guest_chain, host_chain],
    input_variables=['transcript'],
    output_variables=["summary", "guest_post", "self_post"],
    verbose=True)

results = overall_chain({"transcript": "Your podcast transcript here"})

# Print the results
print(results)



> Entering new SequentialChain chain...

> Finished chain.
{'transcript': 'Your podcast transcript here', 'summary': 'Response 1', 'guest_post': 'Response 2', 'self_post': 'Response 3'}


## Adding parallelism

In [ ]:

from langchain.schema.runnable import RunnableParallel
map_chain = RunnableParallel(summary=summary_chain, guest=guest_chain, host=host_chain)
map_chain.invoke({"transcript": "Your podcast transcript here"})

In [3]:
from langchain.schema import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms.fake import FakeListLLM
from langchain.chains.sequential import SequentialChain
from langchain.schema.runnable import RunnablePassthrough

# LLM to use in each stage 
responses = ["Response 1", "Response 2", "Response 3"]
llm = FakeListLLM(responses=responses)

synopsis_prompt = PromptTemplate(
    input_variables=['title'],
    template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
)

review_prompt = PromptTemplate(
    input_variables=['synopsis'],
    template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
)

# chain = (
#     {"synopsis": synopsis_prompt | llm | StrOutputParser()}
#     | review_prompt
#     | llm
#     | StrOutputParser()
# )
# chain.invoke({"title": "Tragedy at sunset on the beach"})


synopsis_chain = synopsis_prompt | llm | {"synopsis": RunnablePassthrough()}
review_chain = review_prompt | llm  | {"synopsis": RunnablePassthrough(), "synopsidds": RunnablePassthrough()}
chain = synopsis_chain |  review_chain
out = chain.invoke({"title": "Tragedy at sunset on the beach"})
out

{'synopsis': 'Response 2', 'synopsidds': 'Response 2'}

In [22]:
from langchain.schema.runnable import RunnablePassthrough

def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

runnable = {
    'llm1':  fake_llm,
    'llm2':  fake_llm,
} | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
  )

runnable.invoke('hello')

AttributeError: type object 'RunnablePassthrough' has no attribute 'assign'

In [1]:
doc = """
[INTRO MUSIC]

Host 1: (Warm and welcoming tone) Hello, everyone, and welcome to [Your Podcast Name], the show where we [briefly describe the podcast's focus or theme].

Host 2: (Energetic) I'm [Your Name], and I'm [Co-Host's Name], and we're thrilled to have you here with us today!

Host 1: (Enthusiastic) That's right! In today's episode, we have a fantastic lineup for you. We'll be discussing [Briefly introduce the main topic or theme of the episode].

Host 2: (Curious) And we have a very special guest joining us, [Guest's Name], who is an expert in [Guest's Expertise]. We can't wait to dive into the conversation!

Host 1: (Engaging) But before we get started, a quick reminder to subscribe to our podcast if you haven't already, so you never miss an episode. And if you enjoy our show, please leave us a review on your favorite podcast platform.

[SEGUE MUSIC]

Host 2: (Transition) Alright, let's jump into today's topic. [Ask a question or set the stage for the discussion].

Host 1: (Add relevant information) Yes, and I think it's important to mention [Add key points or interesting facts related to the topic].

[DISCUSSION]

[Podcast content: Engage in a thoughtful and engaging discussion, share stories, and exchange ideas with your guest.]

[COMMERCIAL BREAK]

Host 2: (Promotion) Before we continue, a quick word from our sponsors.

[Insert a brief commercial or promotional message.]

[OUTRO MUSIC]

Host 1: (Closing) And that's a wrap for today's episode of [Your Podcast Name]! We hope you enjoyed the discussion as much as we did.

Host 2: (Thank you message) Thank you, [Guest's Name], for joining us today. Your insights were invaluable.

Host 1: (Call to action) As always, if you have any feedback, questions, or topics you'd like us to cover in future episodes, don't hesitate to reach out to us on our social media or website.

Host 2: (Farewell) Until next time, this is [Your Name] and [Co-Host's Name] signing off. Stay curious and keep exploring!

[OUTRO MUSIC]
"""